In [ ]:
import json
import time
import requests
import telepot
from telepot.loop import MessageLoop
from influxdb import InfluxDBClient

In [ ]:
class NotificationBot:
    def __init__(self, settings):
        self.settings = settings
        self.token = settings["telegramToken"]
        
        # InfluxDB Setup
        self.influx_client = InfluxDBClient(
            host=settings["influx"]["host"],
            port=settings["influx"]["port"],
            database=settings["influx"]["db"],
            username=settings["ifnlux"]["username"],
            password=settings["ifnlux"]["password"]
        )

        # Config mapping for zones and comfort band (20-25°C from paper)
        self.zones = ["Tin_Zone1", "Tin_Zone2", "Tin_Zone4"]
        # Map zones to their corresponding shade status field in InfluxDB
        self.shades = {
            "Zone 1": ["ShadeStatus_Zone1_Wall2", "ShadeStatus_Zone1_Wall8", "ShadeStatus_Zone1_Wall9"],
            "Zone 2": ["ShadeStatus_Zone2_Wall2", "ShadeStatus_Zone2_wall3"],
            "Zone 4": ["ShadeStatus_Zone4_Wall2"]
        }
        self.PE_GAS = settings["factors"]["PE_GAS"]
        self.PE_ELEC = settings["factors"]["PE_ELEC"]
        self.COP_H = settings["factors"]["COP_H"]
        self.EER_C = settings["factors"]["EER_C"]

        self.bot = telepot.Bot(self.token)
        MessageLoop(self.bot, {'chat': self.on_chat_message}).run_as_thread()

    def on_chat_message(self, msg):
        content_type, chat_type, chat_id = telepot.glance(msg)
        text = msg.get('text', '').strip().lower()

        if text == '/temperature':
            self.send_temperature_status(chat_id)
        elif text == '/shades':
            self.send_shading_status(chat_id)
        elif text == '/energy':
            self.send_energy_report(chat_id)
        elif text == '/help':
            msg = ("Commands:\n"
                "/temperature - Get temperature for all zones\n"
                "/shades - Get current blind positions\n",
                "/energy - Get current consumption and primary energy")
            self.bot.sendMessage(chat_id, msg)

    def send_temperature_status(self, chat_id):
        """Fetch latest temperatures for all zones."""
        try:
            # Query all zone temperatures at once
            fields = ", ".join([f'last("{zone}") as {zone}' for zone in self.zones])
            query = f'SELECT {fields} FROM "bms_data"'
            result = self.influx_client.query(query)
            points = list(result.get_points())

            if points:
                data = points[0]
                status_msg = "🌡 **Current Temperatures:**\n"
                for zone in self.zones:
                    temp = data[zone]
                    status_msg += f"- {zone.replace('Tin_', '')}: {temp:.1f}°C\n"
                self.bot.sendMessage(chat_id, status_msg, parse_mode='Markdown')
            else:
                self.bot.sendMessage(chat_id, "No temperature data found.")
        except Exception as e:
            self.bot.sendMessage(chat_id, f"Error: {e}")

    def send_shading_status(self, chat_id):
        """Check if blinds are Open (0) or Closed (7)."""
        try:
            # Create a flat list of all field names to query InfluxDB
            all_fields = [field for sublist in self.shades.values() for field in sublist]

            # Build the query string
            query_fields = ", ".join([f'last("{f}") as "{f}"' for f in all_fields])
            query = f'SELECT {query_fields} FROM "bms_data"'
            result = self.influx_client.query(query)
            points = list(result.get_points())

            if points:
                data = points[0]
                msg = "🪟 **Shading Status:**\n"
                for zone, fields in self.shades.items():
                    msg += f"🏠 **{zone}**\n"
                    for f in fields:
                        status_val = data.get(f, 0.0)
                        # Displaying 7.0 as Closed and 0.0 as Open per FMU logic
                        icon = "🌑" if status_val > 1.0 else "☀️"
                        text = "Closed" if status_val > 1.0 else "Open"
                        
                        # Clean up field name for display (e.g., ShadeStatus_Zone1_Wall2 -> Wall 2)
                        label = f.split('_')[-1]
                        msg += f"  - {label}: {text} {icon}\n"
                    msg += "\n"
                self.bot.sendMessage(chat_id, msg, parse_mode='Markdown')
            else:
                self.bot.sendMessage(chat_id, "No shading data found.")
        except Exception as e:
            self.bot.sendMessage(chat_id, f"Shading error: {e}")

    def send_energy_report(self, chat_id):
        """Fetch individual energy components and calculate total primary energy."""
        try:
            # Query the latest values for the three components
            query = 'SELECT last("Electricity") as elec, last("DistrictHeating") as heat, last("DistrictCooling") as cool FROM "bms_data"'
            result = self.influx_client.query(query)
            points = list(result.get_points())

            if points:
                # Convert from Joules to kWh
                data = points[0]
                elec = data['elec'] / 3600000 if data['elec'] else 0.0
                heat = data['heat'] / 3600000 if data['heat'] else 0.0
                cool = data['cool'] / 3600000 if data['cool'] else 0.0

                # Calculate Primary Energy (PE) using the specific config factors
                pe_heat = (heat / self.settings["factors"]["COP_H"]) * self.settings["factors"]["PE_GAS"]
                pe_cool = (cool / self.settings["factors"]["EER_C"]) * self.settings["factors"]["PE_ELEC"]
                pe_elec = elec * self.settings["factors"]["PE_ELEC"]
                
                total_primary = pe_heat + pe_cool + pe_elec

                # Format and Send the message
                msg = "⚡ **Energy Consumption Report**\n\n"
                msg += f"🔌 **Electricity:** {elec:.2f} kWh\n"
                msg += f"🔥 **Heating:** {heat:.2f} kWh\n"
                msg += f"❄️ **Cooling:** {cool:.2f} kWh\n"
                msg += "--- \n"
                msg += f"🌍 **Total Primary Energy:** {total_primary:.2f} kWh_pe\n"
                
                self.bot.sendMessage(chat_id, msg, parse_mode='Markdown')
            else:
                self.bot.sendMessage(chat_id, "No energy data found.")
        except Exception as e:
            print(f"Energy report error: {e}")
            self.bot.sendMessage(chat_id, "Error calculating energy data.")

    def monitor_and_alert(self, chat_id):
        """Check all zones for comfort violations (20-25°C)."""
        try:
            fields = ", ".join([f'last("{z}") as {z}' for z in self.zones])
            query = f'SELECT {fields} FROM "bms_data"'
            result = self.influx_client.query(query)
            points = list(result.get_points())

            if points:
                data = points[0]
                for z in self.zones:
                    temp = data[z]
                    if temp < 20.0 or temp > 25.0:
                        zone_label = z.replace('Tin_', '')
                        alert_msg = f"⚠️ ALERT: {zone_label} is {temp:.1f}°C! (Comfort range 20-25°C)"
                        self.bot.sendMessage(chat_id, alert_msg)
        except Exception as e:
            print(f"Monitor error: {e}")

In [ ]:
with open("settings.json") as f:
    conf = json.load(f)

bot = NotificationBot(conf)
print("Notification Bot started...")

TEST_CHAT_ID = 7201915868

try:
    while True:
        # Run the monitoring check every 60 seconds
        bot.monitor_and_alert(TEST_CHAT_ID)
        time.sleep(60)
except KeyboardInterrupt:
    print("Stopping Bot...")